In [1]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression, Ridge


In [2]:
p = 7
n = 10
min_cor = 0
max_cor = 0.1

true_betas = np.array([0,1,2,3,4,5,6])

In [3]:
def get_sim_data(p, n, min_cor, max_cor, true_betas):
    
    sd_vec = np.ones(p) 
    mean = np.zeros(p)
    cor_matrix = np.zeros((p,p))

    correlation = np.random.uniform(min_cor, max_cor, int(p * (p - 1) / 2))
    cor_matrix[np.triu_indices(p, 1)] = correlation
    cor_matrix[np.tril_indices(p, -1)] = cor_matrix.T[np.tril_indices(p, -1)]
    np.fill_diagonal(cor_matrix, 1)


    D = np.diag(sd_vec)
    sigma = D.dot(cor_matrix).dot(D)

    X = np.random.multivariate_normal(mean, sigma, n)
    eps = np.random.normal(0, 1, n)

    y = X.dot(true_betas) + eps 
    
    y = pd.Series(y, name = "y")
    
    column_names = []
    
    for value in range(1, p + 1): 
        
        column = f"X_{value}"
        column_names.append(column)
        
    
    X = pd.DataFrame(X, columns = column_names)
    
    df = pd.concat([y, X], axis = 1)
    
    return y, X, df



In [4]:
y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas)

In [5]:
df

,y,X_1,X_2,X_3,X_4,X_5,X_6,X_7
0,-9.663170,0.739373,-1.091106,-1.297383,-0.555529,-0.125290,0.229336,-0.800338
1,3.232785,-1.993455,-1.340517,1.352472,-0.507367,-1.109876,0.812955,0.792466
2,2.307245,-0.843593,1.504612,-2.908839,2.076752,0.011844,0.285348,-0.152057
3,-7.365547,0.075458,-0.802131,1.077076,-0.596524,0.217227,0.092125,-1.367232
4,-3.562954,0.790314,0.421671,0.666817,-0.031308,0.961252,0.655175,-2.256460
5,-7.546910,-0.904204,0.545975,0.172877,1.156073,-1.013156,-0.739748,-0.591381
6,-6.656034,-2.377774,-0.024327,-0.209977,-0.799213,-0.008274,0.765994,-1.312374
7,4.497663,0.804676,-0.574149,-0.289605,1.778267,-0.896669,-0.176737,0.901017
8,-1.514366,-0.549130,-0.277393,1.102904,0.455846,-2.150165,-0.000121,0.605223
9,-7.116216,-0.051931,-0.702834,-0.392091,-1.257349,-1.845552,0.530665,0.485135


In [6]:
reg = LinearRegression().fit(X, y)

In [7]:
reg.coef_


array([0.25711778, 0.87592911, 1.97985535, 3.55056218, 2.93626882,
       6.43934611, 4.75873918])

In [8]:
"""Zero multicollinearity.
   n = 200; p = 2, 5, 50, 100, 150, 200, 210, 250. 
"""
np.random.seed(123)

n_sim = 30
p_sim = np.array([2, 28, 30, 50])
min_corr_sim = 0
max_corr_sim = 0 
iterations_sim = 10000

for p in p_sim: 
    
    matrix_var = []
    matrix_betas = []
    
    for i in range(iterations_sim):
    
        true_betas_sim = np.repeat(5, p)
    
        y, X, df = get_sim_data(p, n_sim, min_corr_sim, max_corr_sim, true_betas_sim) 
    
        var_cov = np.linalg.inv(np.dot(X.T, X))
        var_betas = var_cov.diagonal() 
        
        #reg = LinearRegression().fit(X, y)
        betas = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
        
        #matrix_betas.append(reg.coef_)
        matrix_betas.append(betas)
        
        matrix_var.append(var_betas)
        
        X_names = []
        beta_names = []
    
        for value in range(1, p + 1): 
            
            column_X = f"X_{value}"
            column_betas = f"beta_{value}"
            X_names.append(column_X)
            beta_names.append(column_betas)
        
        df_var = pd.DataFrame(matrix_var, columns = X_names)
        df_betas = pd.DataFrame(matrix_betas, columns = beta_names) 
        
        bias_squared = (df_betas.mean() - true_betas_sim)**2
        
    #print(df_var.mean()) 
    #print(df_betas.mean())
    print(bias_squared)
    #print(df_var)

beta_1    0.000010
beta_2    0.000018
dtype: float64
beta_1    1.153272e-05
beta_2    3.689798e-05
beta_3    1.395766e-06
beta_4    9.719223e-06
beta_5    2.850345e-07
dtype: float64
beta_1     8.522421e-06
beta_2     8.232463e-06
beta_3     1.891248e-06
beta_4     2.607985e-05
beta_5     2.423444e-05
beta_6     1.212774e-05
beta_7     1.556953e-06
beta_8     7.594101e-06
beta_9     1.144627e-09
beta_10    4.671491e-06
dtype: float64
beta_1     1.150142e-05
beta_2     3.193251e-05
beta_3     6.984134e-06
beta_4     1.025836e-05
beta_5     5.667166e-06
beta_6     9.554523e-06
beta_7     2.115020e-05
beta_8     3.267928e-07
beta_9     1.570551e-05
beta_10    2.502321e-05
beta_11    3.028083e-05
beta_12    2.036054e-05
beta_13    9.771474e-06
beta_14    5.059418e-05
beta_15    1.035470e-04
dtype: float64
beta_1     2.391025e-06
beta_2     1.076366e-05
beta_3     1.471301e-06
beta_4     5.653493e-05
beta_5     1.636993e-06
beta_6     5.371709e-06
beta_7     5.170249e-07
beta_8     3.136007

In [10]:
np.random.seed(159)

p = 198
n = 200
true_betas = np.repeat(5, p)
min_cor = 0 
max_cor = 0 
alpha = 0.001


y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas)

n, m = X.shape
I = np.identity(m)


ols = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)

ols_var_cov = np.linalg.inv(np.dot(X.T, X))
ols_var = ols_var_cov.diagonal()

alphas = np.array([0, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1, 2, 3, 5, 10, 15, 20])

matr_var = []
matr_beta = []

for a in alphas: 

    ridge_beta = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), X.T), y)
    matr_beta.append(ridge_beta)
    
    ridge_var_cov = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), np.dot(X.T, X)), np.linalg.inv(np.dot(X.T, X) + a * I))
    ridge_var = ridge_var_cov.diagonal()
    matr_var.append(ridge_var)

beta_var_names = []
ridge_beta_names = []
    
for value in range(1, p + 1): 
    
    column_betas_var = f"beta_var_{value}"
    column_betas = f"beta_{value}"
    beta_var_names.append(column_betas_var)
    ridge_beta_names.append(column_betas)


In [11]:
beta_var = pd.DataFrame(matr_var, columns = beta_var_names)
beta_var["alpha"] = alphas
beta_var.set_index("alpha")    

,beta_var_1,beta_var_2,beta_var_3,beta_var_4,beta_var_5,beta_var_6,beta_var_7,beta_var_8,beta_var_9,beta_var_10,...,beta_var_189,beta_var_190,beta_var_191,beta_var_192,beta_var_193,beta_var_194,beta_var_195,beta_var_196,beta_var_197,beta_var_198
alpha,,,,,,,,,,,,,,,,,,,,,
0.0000,2.427521,0.663690,1.027757,1.741210,8.652040,0.705819,1.167791,0.209445,0.475403,1.731693,...,0.464720,1.398587,0.242995,4.707146,5.263580,0.286077,0.432695,0.264678,2.052473,2.387135
0.0001,2.268247,0.638268,1.015365,1.677230,8.118124,0.696873,1.102453,0.208107,0.460090,1.643960,...,0.463072,1.368871,0.240256,4.421376,4.932370,0.284450,0.421808,0.254588,1.924919,2.358828
0.0010,1.365506,0.489142,0.920092,1.289592,5.063371,0.636114,0.727835,0.198151,0.372755,1.134611,...,0.451391,1.173477,0.223555,2.786644,3.046089,0.271885,0.354129,0.194701,1.197613,2.140829
0.0100,0.233427,0.236424,0.484755,0.491798,0.885691,0.419897,0.204628,0.153566,0.250316,0.344555,...,0.389761,0.584647,0.184876,0.552214,0.567427,0.210356,0.193110,0.085665,0.232947,1.135968
0.1000,0.071878,0.109102,0.109703,0.102415,0.102458,0.134093,0.083359,0.087886,0.147321,0.092087,...,0.171041,0.131156,0.122357,0.108965,0.119170,0.112952,0.080600,0.037393,0.067317,0.221369
0.2000,0.052158,0.083912,0.072989,0.070817,0.071605,0.079757,0.069978,0.068406,0.105920,0.066938,...,0.106042,0.082435,0.096294,0.079727,0.078798,0.084015,0.060815,0.032334,0.055253,0.124536
0.5000,0.033333,0.055552,0.044977,0.045025,0.047987,0.041008,0.053829,0.046834,0.062048,0.044307,...,0.054870,0.048374,0.064342,0.053194,0.045295,0.051499,0.040217,0.026727,0.040510,0.059026
1.0000,0.024475,0.038056,0.032094,0.032398,0.033820,0.026810,0.041855,0.034336,0.040391,0.032550,...,0.035335,0.034211,0.044444,0.037953,0.030069,0.033482,0.029148,0.022248,0.029858,0.035925
2.0000,0.018267,0.024648,0.022964,0.023233,0.022680,0.018520,0.030327,0.024222,0.026329,0.023708,...,0.023922,0.024243,0.028791,0.026096,0.020006,0.021458,0.021236,0.017511,0.020976,0.023199


In [12]:
ridge_beta = pd.DataFrame(matr_beta, columns = ridge_beta_names)
ridge_beta["alpha"] = alphas
ridge_beta.set_index("alpha")

,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6,beta_7,beta_8,beta_9,beta_10,...,beta_189,beta_190,beta_191,beta_192,beta_193,beta_194,beta_195,beta_196,beta_197,beta_198
alpha,,,,,,,,,,,,,,,,,,,,,
0.0000,1.431469,5.807467,3.926156,2.236071,0.072503,7.624101,7.220508,5.102859,4.031463,1.447353,...,4.220671,5.882952,5.872391,8.546182,1.462723,4.922933,4.285627,4.374181,1.437395,5.958386
0.0001,1.551141,5.760691,3.915839,2.308962,0.281415,7.600136,7.147344,5.096939,4.067559,1.535833,...,4.228182,5.847133,5.857172,8.392787,1.628563,4.918490,4.315292,4.401010,1.545047,5.926595
0.0010,2.338443,5.451785,3.858992,2.793782,1.640578,7.438045,6.670658,5.058287,4.303149,2.121703,...,4.274300,5.623312,5.755458,7.393551,2.708076,4.891785,4.513975,4.574284,2.255773,5.701470
0.0100,4.108101,4.753317,3.948965,3.958287,4.418631,6.995541,5.674130,4.953750,4.783558,3.498915,...,4.297046,5.352289,5.495505,5.317081,4.910800,4.871660,5.022244,4.908232,3.901663,4.875434
0.1000,4.758536,4.820550,4.730247,4.126283,4.941148,6.713544,5.227479,4.417448,4.578426,3.955284,...,3.730093,6.021328,5.257863,4.616056,5.022495,4.828649,5.279141,4.975867,4.645925,3.368100
0.2000,4.744429,5.063842,5.059990,3.865562,4.874784,6.685456,5.036722,4.032856,4.343217,3.838261,...,3.357149,6.355777,5.131745,4.428845,4.718949,4.763723,5.262315,4.970348,4.674205,2.772379
0.5000,4.603572,5.383110,5.408980,3.515787,4.777159,6.622313,4.653210,3.432380,4.030708,3.569397,...,2.814711,6.746293,4.826490,4.070268,4.211372,4.675612,5.206396,4.864943,4.557786,2.034509
1.0000,4.401496,5.517441,5.474450,3.388484,4.722273,6.515898,4.239832,2.938190,3.873702,3.321878,...,2.446932,6.900269,4.468958,3.700577,3.778271,4.617793,5.180779,4.629816,4.349650,1.661489
2.0000,4.120503,5.530916,5.266060,3.435371,4.661601,6.315323,3.701556,2.370742,3.801054,3.054539,...,2.126772,6.891306,4.039137,3.245606,3.289118,4.547337,5.215155,4.207783,4.066485,1.514109


$$
V\left(\hat{\boldsymbol{\beta}}^{*}\right)=\sigma^{2}\left[\mathbf{X}^{\prime} \mathbf{X}+k \mathbf{I}\right]^{-1}\left(\mathbf{X}^{\prime} \mathbf{X}\right)\left[\mathbf{X}^{\prime} \mathbf{X}+k \mathbf{I}\right]^{-1}.
$$

In [13]:
np.random.seed(159)

p = 198
n = 200
min_cor = 0 
max_cor = 0 
iterations_sim = 5


alphas = np.array([0, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1, 2, 3, 5, 10, 15, 20])
#alphas = np.array([0, 0.0001])


for a in alphas: 
    
    matr_var = []
    matr_beta = []
    
    for i in range(iterations_sim):
        
        true_betas_sim = np.repeat(5, p)
    
        y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas_sim) 
        
        n, m = X.shape
        I = np.identity(m)

        ridge_beta = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), X.T), y)
        matr_beta.append(ridge_beta)
        df_ridge_betas = pd.DataFrame(matr_beta).mean()
        
        ridge_var_cov = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), np.dot(X.T, X)), np.linalg.inv(np.dot(X.T, X) + a * I))
        ridge_var = ridge_var_cov.diagonal()
        matr_var.append(ridge_var)
        df_ridge_var = pd.DataFrame(matr_var).mean()
        
    
    print(df_ridge_betas)
    print(df_ridge_var)
       
    
        

    #beta_var_names = []
    #ridge_beta_names = []
    
    #for value in range(1, p + 1): 
    
        #column_betas_var = f"beta_var_{value}"
        #column_betas = f"beta_{value}"
        #beta_var_names.append(column_betas_var)
        #ridge_beta_names.append(column_betas)




0      4.635122
1      5.729616
2      5.058337
3      4.428545
4      3.234887
         ...   
193    6.171077
194    5.741595
195    4.427468
196    4.298631
197    4.852898
Length: 198, dtype: float64
0      1.007912
1      0.496712
2      0.474403
3      0.850550
4      2.408572
         ...   
193    1.491216
194    0.654407
195    0.566143
196    1.249935
197    1.001389
Length: 198, dtype: float64
0      4.335456
1      5.466356
2      5.149386
3      4.969014
4      5.489306
         ...   
193    4.835337
194    5.127192
195    4.673648
196    5.121645
197    4.933246
Length: 198, dtype: float64
0      2.327728
1      0.707047
2      0.931330
3      0.943510
4      0.693665
         ...   
193    1.358048
194    1.027283
195    0.443578
196    0.779883
197    0.921927
Length: 198, dtype: float64
0      4.623623
1      5.046872
2      4.901863
3      5.304393
4      4.748583
         ...   
193    5.047025
194    5.340651
195    5.004217
196    5.323487
197    4.973445
Length: 

In [14]:
#pd.DataFrame(matr_beta)